In [1]:
import pandas as pd
import opendatasets as od

od.download("https://www.kaggle.com/competitions/electricity-consumption")
df = pd.read_csv("./electricity-consumption/train.csv")


Skipping, found downloaded files in ".\electricity-consumption" (use force=True to force download)


In [3]:
import pandas as pd
import numpy as np
import math
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler

class TimeFeaturesExtractor(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self  # Мы не нуждаемся в обучении для этой трансформации

    def transform(self, X):
        # Извлекаем временные признаки
        X['year'] = X['datetime'].dt.year
        X['month'] = X['datetime'].dt.month
        X['season'] = X['datetime'].dt.quarter
        X['day'] = X['datetime'].dt.day
        X['hour'] = X['datetime'].dt.hour
        X['dayofyear'] = X['datetime'].dt.day_of_year
        X['dayofweek'] = X['datetime'].dt.day_of_week
        X['is_weekend'] = X['dayofweek'].isin([5, 6]).astype(int)

        # Убираем колонку 'datetime' после обработки
        X = X.drop(columns=['datetime'])

        return X

In [4]:
class TrigonometricFeaturesExtractor(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        # Добавляем тригонометрические признаки для временных признаков
        X['sin_month'] = X['month'].apply(math.sin)
        X['cos_month'] = X['month'].apply(math.cos)

        X['sin_hour'] = X['hour'].apply(math.sin)
        X['cos_hour'] = X['hour'].apply(math.cos)

        X['hour_sin'] = (X['hour'] / 23 * 2 * np.pi).apply(math.sin)
        X['hour_cos'] = (X['hour'] / 23 * 2 * np.pi).apply(math.cos)

        X['month_sin'] = ((X['month'] - 1) / 11 * 2 * np.pi).apply(math.sin)
        X['month_cos'] = ((X['month'] - 1) / 11 * 2 * np.pi).apply(math.cos)

        # Преобразуем дату в индекс, если нужно для дальнейшего использования
        day = 24
        year = 365.2425 * day

        X['day_sin'] = (X['hour'] * 2 * np.pi / day).apply(math.sin)
        X['day_cos'] = (X['hour'] * 2 * np.pi / day).apply(math.cos)

        X['year_sin'] = (X['hour'] * 2 * np.pi / year).apply(math.sin)
        X['year_cos'] = (X['hour'] * 2 * np.pi / year).apply(math.cos)

        return X

In [5]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import pandas as pd

pipeline_preprocessing = Pipeline([
    ('time_features', TimeFeaturesExtractor()), # призанки даты и времени
    ('trigonometric_features', TrigonometricFeaturesExtractor()), # Тригонометрические признаки
])

# просто удаляем пропуски
df = df[~df['total'].isna()]
df['total'] = df['total'].astype('int64')

# Преобразуем строковый столбец 'datetime' в формат datetime
df['datetime'] = pd.to_datetime(df['datetime'], format="%d.%m.%Y %H:%M:%S")

# Разделяем данные на train и test, извлекая год прямо из 'datetime'
train = df[df['datetime'].dt.year != 2008]
test = df[df['datetime'].dt.year == 2008]

# Удалим ненужные колонки (например, целевую переменную) из обучающих данных
X_train = train.drop(columns=['total'])
y_train = train['total']

# Применим пайплайн и обучим модель
pipeline_preprocessing.fit(X_train, y_train)

# Удалим ненужные колонки из тестовых данных
X_test = test.drop(columns=['total'])
y_test = test['total']

# Применим пайплайн к тестовым данным (переход от fit к transform)
X_train_transformed = pipeline_preprocessing.transform(X_train)
X_test_transformed = pipeline_preprocessing.transform(X_test)

In [6]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

# Полный пайплайн с моделью
full_pipeline = Pipeline([
    ('preprocessing', pipeline_preprocessing),  # Готовый пайплайн для обработки данных
    ('scaler', StandardScaler()),               # Масштабирование данных
    ('model', RandomForestRegressor())          # Заглушка для модели
])

In [7]:
# Сетка гиперпараметров для Random Forest
param_grid = {
    'model__n_estimators': [50, 100, 150],
    'model__max_depth': [3, 5, 7],
    'model__min_samples_split': [2, 5],
    'model__min_samples_leaf': [1, 2]
}

In [8]:
# Подбор гиперпараметров через GridSearchCV
search = GridSearchCV(full_pipeline,
                      param_grid,
                      cv=5,
                      scoring='neg_mean_squared_error')
# Обучаем алгоритм подбора гиперпараметров
search.fit(X_train, y_train)

# Лучшие параметры
print(f"Best params: {search.best_params_}")
print(f"Best score: {search.best_score_}")

Best params: {'model__max_depth': 7, 'model__min_samples_leaf': 2, 'model__min_samples_split': 2, 'model__n_estimators': 100}
Best score: -209495062.6417386


In [10]:
best_model = search.best_estimator_

In [12]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

y_pred = best_model.predict(X_test)
print("Test MSE:", mean_squared_error(y_test, y_pred))

Test MSE: 306241642.0240623


In [14]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV

# Определяем модель
model = RandomForestClassifier()

# Создаем объект RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=full_pipeline,
    param_distributions=param_grid,  # Диапазон параметров
    n_iter=20,                       # 20 случайных комбинаций
    cv=5,                            # 5-кратная кросс-валидация
    scoring='accuracy',              # Метрика - точность
    n_jobs=-1,                       # Параллельное выполнение
    random_state=42                 # Фиксируем сид для воспроизводимости
)

In [15]:
random_search.fit(X_train, y_train)

c:\Users\dimac\anaconda3\Lib\site-packages\sklearn\model_selection\_search.py:1051: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan]
  warnings.warn(


RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('preprocessing',
                                              Pipeline(steps=[('time_features',
                                                               TimeFeaturesExtractor()),
                                                              ('trigonometric_features',
                                                               TrigonometricFeaturesExtractor())])),
                                             ('scaler', StandardScaler()),
                                             ('model',
                                              RandomForestRegressor())]),
                   n_iter=20, n_jobs=-1,
                   param_distributions={'model__max_depth': [3, 5, 7],
                                        'model__min_samples_leaf': [1, 2],
                                        'model__min_samples_split': [2, 5],
                                        'model__n_estimators': [50, 100, 150]},
                   random_state=42, scoring='accuracy')

In [16]:
random_search.best_params_

{'model__n_estimators': 150,
 'model__min_samples_split': 5,
 'model__min_samples_leaf': 2,
 'model__max_depth': 7}

In [17]:
best_model = random_search.best_estimator_
y_pred = best_model.predict(X_test)
print("Test MSE:", mean_squared_error(y_test, y_pred))

Test MSE: 303437381.0619479


In [20]:
!pip install scikit-optimize

   ---------------------------------------- 0.0/107.8 kB ? eta -:--:--
   --- ------------------------------------ 10.2/107.8 kB ? eta -:--:--
   ------------------ -------------------- 51.2/107.8 kB 660.6 kB/s eta 0:00:01
   -------------------------------------- 107.8/107.8 kB 895.0 kB/s eta 0:00:00


In [21]:
from skopt import BayesSearchCV
from sklearn.ensemble import RandomForestClassifier

# Определяем пространство гиперпараметров
param_space = {
    'n_estimators': (10, 200),                  # Количество деревьев (диапазон значений)
    'max_depth': (1, 20),                       # Глубина дерева (диапазон значений)
    'min_samples_split': (2, 10),               # Минимум элементов для разбиения узла
    'min_samples_leaf': (1, 5)                  # Минимум элементов в листе
}

# Создаем модель
model = RandomForestClassifier()

# Оптимизация с помощью Байесовского поиска
bayes_search = BayesSearchCV(
    estimator=model,
    search_spaces=param_space,     # Пространство гиперпараметров
    n_iter=20,                     # Количество итераций
    cv=5,                          # Количество фолдов для кросс-валидации
    scoring='accuracy',            # Метрика качества
    n_jobs=-1,                     # Параллельное выполнение
    random_state=42
)

In [22]:
bayes_search.fit(X_train, y_train)


c:\Users\dimac\anaconda3\Lib\site-packages\sklearn\model_selection\_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


DTypePromotionError: The DType <class 'numpy.dtypes.DateTime64DType'> could not be promoted by <class 'numpy.dtypes.Int32DType'>. This means that no common DType exists for the given inputs. For example they cannot be stored in a single array unless the dtype is `object`. The full list of DTypes is: (<class 'numpy.dtypes.DateTime64DType'>, <class 'numpy.dtypes.Int32DType'>, <class 'numpy.dtypes.Int32DType'>, <class 'numpy.dtypes.Int32DType'>, <class 'numpy.dtypes.Int32DType'>, <class 'numpy.dtypes.Int32DType'>, <class 'numpy.dtypes.Int32DType'>, <class 'numpy.dtypes.Int32DType'>, <class 'numpy.dtypes.Int32DType'>)